In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [20]:
def convert_to_serializable(obj):
    """Convert numpy types to Python native types for JSON serialization."""
    if isinstance(obj, (np.int_, np.intc, np.intp, np.int8,
        np.int16, np.int32, np.int64, np.uint8,
        np.uint16, np.uint32, np.uint64)):
        return int(obj)
    elif isinstance(obj, (np.float_, np.float16, np.float32, np.float64)):
        return float(obj)
    elif isinstance(obj, (np.ndarray,)):
        return obj.tolist()
    elif isinstance(obj, dict):
        return {str(k): convert_to_serializable(v) for k, v in obj.items()}
    elif isinstance(obj, (list, tuple)):
        return [convert_to_serializable(item) for item in obj]
    return obj

In [4]:
fp = '../Data/Cleaned_Mel_CPS_19_Dataset.csv'
data = pd.read_csv(fp)

In [5]:
data

,Unnamed: 0,Coder,session_id,mellon_id,nickname,event_result,KC,task,Primary code,event_star,...,female_members,female_maj,gender,ethnicity,application_status,first_generation,urm,first_language,birth_year,birth_month
0,1,Rohin,7298_PartyVenue,173442,Liz,what's up lads,6,PartyVenue,SMC,9/28/19,...,4,1,F,Unknown / declined to state,Sophomore,0,0,English only,2001.0,2.0
1,2,Rohin,7298_PartyVenue,202618,bean boi,hello,6,PartyVenue,SMC,9/28/19,...,4,1,F,Asian / Asian American,Freshman,1,0,Non-English,2001.0,2.0
2,3,Rohin,7298_PartyVenue,163093,Alice,hi its too early for this,6,PartyVenue,SMC,9/28/19,...,4,1,F,Asian / Asian American,Freshman,1,0,English/non-English,2001.0,3.0
3,4,Rohin,7298_PartyVenue,173442,Liz,bean boi i like ur name,6,PartyVenue,SMC,9/28/19,...,4,1,F,Unknown / declined to state,Sophomore,0,0,English only,2001.0,2.0
4,5,Rohin,7298_PartyVenue,202618,bean boi,thanks,6,PartyVenue,SMC,9/28/19,...,4,1,F,Asian / Asian American,Freshman,1,0,Non-English,2001.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4162,4163,Blair,7329_candidate,199244,Paige,but C seemed chill,1,candidate,SSI,9/28/19,...,3,1,F,Hispanic,Freshman,0,1,English only,2001.0,3.0
4163,4164,Blair,7329_candidate,190611,kyle,you guys want to submit CBA ?,5,candidate,CM,9/28/19,...,3,1,M,Hispanic,Senior,0,1,English only,1998.0,10.0
4164,4165,Blair,7329_candidate,173410,shiny,i'm down,5,candidate,SN,9/28/19,...,3,1,F,Asian / Asian American,Freshman,1,0,Non-English,2001.0,9.0
4165,4166,Blair,7329_candidate,199244,Paige,Ya,5,candidate,SN,9/28/19,...,3,1,F,Hispanic,Freshman,0,1,English only,2001.0,3.0


In [18]:
# Import required ML libraries
from sklearn.model_selection import KFold, cross_val_score, cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, confusion_matrix, make_scorer, accuracy_score, f1_score
import numpy as np
import os
import json
from datetime import datetime
import pandas as pd

# Prepare the data
X = data['event_result']
y = data['KC']

# Create TF-IDF features for the full dataset
vectorizer = TfidfVectorizer(ngram_range=(1, 2), lowercase=True)
X_tfidf = vectorizer.fit_transform(X)

# Initialize K-fold cross validation
k_folds = 5
kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)

# Initialize models
models = {
    'Logistic_Regression': LogisticRegression(max_iter=1000),
    'Random_Forest': RandomForestClassifier(n_estimators=100),
    'Naive_Bayes': MultinomialNB(),
    'SVM': LinearSVC(max_iter=2000)
}

# Define scoring metrics
scoring = {
    'accuracy': 'accuracy',
    'f1_macro': 'f1_macro',
    'f1_weighted': 'f1_weighted'
}

# Create Results directory structure
results_dir = '../Results/traditional_ml/unweighted'
run_timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
run_dir = os.path.join(results_dir, f'run_{run_timestamp}')

# Create all necessary subdirectories
subdirs = [
    'metrics/confusion_matrices',
    'metrics/classification_reports',
    'metrics/summary',
    'models/weights',
    'models/configs',
    'visualizations/feature_importance',
    'logs/training_logs'
]
for subdir in subdirs:
    os.makedirs(os.path.join(run_dir, subdir), exist_ok=True)

# Dictionary to store all results
all_results = {}

# Create training log file
log_file = os.path.join(run_dir, 'logs/training_logs/training.log')

# Train and evaluate each model
for name, model in models.items():
    model_log = [f"\n{'='*50}", f"Model: {name}", f"{'='*50}\n"]
    
    # Perform k-fold cross validation with multiple metrics
    cv_results = cross_validate(model, X_tfidf, y, 
                              cv=kf, 
                              scoring=scoring,
                              return_train_score=True)
    
    # Log cross-validation results
    model_log.extend([
        "Cross-validation results:",
        f"Test Accuracy: {cv_results['test_accuracy'].mean():.3f} (+/- {cv_results['test_accuracy'].std() * 2:.3f})",
        f"Test F1 (macro): {cv_results['test_f1_macro'].mean():.3f} (+/- {cv_results['test_f1_macro'].std() * 2:.3f})",
        f"Test F1 (weighted): {cv_results['test_f1_weighted'].mean():.3f} (+/- {cv_results['test_f1_weighted'].std() * 2:.3f})\n"
    ])
    
    # Store predictions and true values for each fold
    predictions = []
    true_values = []
    
    # Train and predict for each fold
    for fold, (train_idx, val_idx) in enumerate(kf.split(X_tfidf), 1):
        X_train_fold = X_tfidf[train_idx]
        X_val_fold = X_tfidf[val_idx]
        y_train_fold = y[train_idx]
        y_val_fold = y[val_idx]
        
        # Train model
        model.fit(X_train_fold, y_train_fold)
        
        # Make predictions
        y_pred_fold = model.predict(X_val_fold)
        
        # Store predictions and true values
        predictions.extend(y_pred_fold)
        true_values.extend(y_val_fold)
        
        # Log fold results
        model_log.extend([
            f"Fold {fold} Results:",
            classification_report(y_val_fold, y_pred_fold)
        ])
    
    # Create and save confusion matrix
    cm = confusion_matrix(true_values, predictions)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f'Confusion Matrix - {name} (All Folds)')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.savefig(os.path.join(run_dir, 'metrics/confusion_matrices', f'{name.lower()}_cm.png'))
    plt.close()

    # Store model results
    model_results = {
        'cross_validation': {
            'accuracy': {'mean': float(cv_results['test_accuracy'].mean()),
                       'std': float(cv_results['test_accuracy'].std())},
            'f1_macro': {'mean': float(cv_results['test_f1_macro'].mean()),
                        'std': float(cv_results['test_f1_macro'].std())},
            'f1_weighted': {'mean': float(cv_results['test_f1_weighted'].mean()),
                          'std': float(cv_results['test_f1_weighted'].std())}
        },
        'confusion_matrix': cm.tolist(),
        'classification_report': classification_report(true_values, predictions, output_dict=True)
    }

    # Feature importance analysis
    if name in ['Logistic_Regression', 'SVM']:
        model.fit(X_tfidf, y)
        feature_names = vectorizer.get_feature_names_out()
        
        feature_importance = {}
        model_log.append(f"\nTop Features by KC for {name}:")
        
        for i, class_label in enumerate(model.classes_):
            coef = model.coef_[i]
            top_indices = np.argsort(coef)[-10:][::-1]
            
            feature_importance[f'KC_{class_label}'] = {
                feature_names[idx]: float(coef[idx])
                for idx in top_indices
            }
            
            model_log.extend([
                f"\nTop features for KC {class_label}:",
                *[f"{feature_names[idx]}: {coef[idx]:.4f}" for idx in top_indices]
            ])
            
        model_results['feature_importance'] = feature_importance
        
        # Save feature importance visualization
        plt.figure(figsize=(12, 8))
        plt.bar(range(len(feature_importance[f'KC_{class_label}'])), 
                list(feature_importance[f'KC_{class_label}'].values()))
        plt.xticks(range(len(feature_importance[f'KC_{class_label}'])), 
                   list(feature_importance[f'KC_{class_label}'].keys()), 
                   rotation=45, ha='right')
        plt.title(f'Top Features Importance - {name}')
        plt.tight_layout()
        plt.savefig(os.path.join(run_dir, 'visualizations/feature_importance', f'{name.lower()}_importance.png'))
        plt.close()
    
    all_results[name] = model_results
    
    # Write model logs
    with open(log_file, 'a') as f:
        f.write('\n'.join(model_log))

# Save all results
# 1. Classification reports
for name, results in all_results.items():
    with open(os.path.join(run_dir, 'metrics/classification_reports', f'{name.lower()}_report.json'), 'w') as f:
        json.dump(results['classification_report'], f, indent=4)

# 2. Summary metrics
summary_data = []
for model_name, results in all_results.items():
    summary_data.append({
        'Model': model_name,
        'Accuracy': f"{results['cross_validation']['accuracy']['mean']:.3f} ± {results['cross_validation']['accuracy']['std']*2:.3f}",
        'F1_Macro': f"{results['cross_validation']['f1_macro']['mean']:.3f} ± {results['cross_validation']['f1_macro']['std']*2:.3f}",
        'F1_Weighted': f"{results['cross_validation']['f1_weighted']['mean']:.3f} ± {results['cross_validation']['f1_weighted']['std']*2:.3f}"
    })

summary_df = pd.DataFrame(summary_data)
summary_df.to_csv(os.path.join(run_dir, 'metrics/summary', 'model_performance.csv'), index=False)

# 3. Full results configuration
with open(os.path.join(run_dir, 'models/configs', 'full_results.json'), 'w') as f:
    json.dump(all_results, f, indent=4)

print(f"\nResults saved in {run_dir}:")
print("├── metrics/")
print("│   ├── confusion_matrices/")
print("│   ├── classification_reports/")
print("│   └── summary/")
print("├── models/")
print("│   └── configs/")
print("├── visualizations/")
print("│   └── feature_importance/")
print("└── logs/")
print("    └── training_logs/")

/Users/amin/Library/CloudStorage/GoogleDrive-masamadi@uci.edu/Shared drives/La(LA)LAB/Research Projects/kc-coding/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/amin/Library/CloudStorage/GoogleDrive-masamadi@uci.edu/Shared drives/La(LA)LAB/Research Projects/kc-coding/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/amin/Library/CloudStorage/GoogleDrive-masamadi@uci.edu/Shared drives/La(LA)LAB/Research Projects/kc-coding/.venv/lib/python3.12/site-packages


Results saved in ../Results/traditional_ml/unweighted/run_20241205_164027:
├── metrics/
│   ├── confusion_matrices/
│   ├── classification_reports/
│   └── summary/
├── models/
│   └── configs/
├── visualizations/
│   └── feature_importance/
└── logs/
    └── training_logs/


In [22]:
# Import required ML libraries
from sklearn.model_selection import KFold, cross_val_score, cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, confusion_matrix, make_scorer, accuracy_score, f1_score
from sklearn.utils.class_weight import compute_class_weight
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import os
import json
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

def convert_to_serializable(obj):
    """Convert numpy types to Python native types for JSON serialization."""
    if isinstance(obj, (np.integer,)):
        return int(obj)
    elif isinstance(obj, (np.floating,)):
        return float(obj)
    elif isinstance(obj, (np.ndarray,)):
        return obj.tolist()
    elif isinstance(obj, dict):
        return {str(k): convert_to_serializable(v) for k, v in obj.items()}
    elif isinstance(obj, (list, tuple)):
        return [convert_to_serializable(item) for item in obj]
    return obj

# Prepare the data
X = data['event_result']
y = data['KC']

# Create TF-IDF features for the full dataset
vectorizer = TfidfVectorizer(ngram_range=(1, 2), lowercase=True)
X_tfidf = vectorizer.fit_transform(X)

# Compute class weights
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y),
    y=y
)
class_weight_dict = dict(zip(np.unique(y), class_weights))

# Initialize K-fold cross validation
k_folds = 5
kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)

# Initialize models with class weights
models = {
    'Logistic_Regression': LogisticRegression(
        max_iter=1000,
        class_weight=class_weight_dict
    ),
    'Random_Forest': RandomForestClassifier(
        n_estimators=100,
        class_weight=class_weight_dict
    ),
    'Naive_Bayes': MultinomialNB(),  # Note: MultinomialNB doesn't support class_weight directly
    'SVM': LinearSVC(
        max_iter=2000,
        class_weight=class_weight_dict
    )
}

# Define scoring metrics
scoring = {
    'accuracy': 'accuracy',
    'f1_macro': 'f1_macro',
    'f1_weighted': 'f1_weighted'
}

# Create Results directory structure for weighted models
results_dir = '../Results/traditional_ml/weighted'
run_timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
run_dir = os.path.join(results_dir, f'run_{run_timestamp}')

# Create all necessary subdirectories
subdirs = [
    'metrics/confusion_matrices',
    'metrics/classification_reports',
    'metrics/summary',
    'models/weights',
    'models/configs',
    'visualizations/feature_importance',
    'logs/training_logs'
]
for subdir in subdirs:
    os.makedirs(os.path.join(run_dir, subdir), exist_ok=True)

# Save class weights configuration
class_weight_config = convert_to_serializable({
    'class_weight_method': 'balanced',
    'class_weights': class_weight_dict
})
with open(os.path.join(run_dir, 'models/configs', 'class_weights.json'), 'w') as f:
    json.dump(class_weight_config, f, indent=4)

# Dictionary to store all results
all_results = {}

# Create training log file
log_file = os.path.join(run_dir, 'logs/training_logs/training.log')

# Log initial setup
with open(log_file, 'w') as f:
    f.write("Training Configuration:\n")
    f.write(f"Timestamp: {run_timestamp}\n")
    f.write(f"Class weights: {json.dumps(convert_to_serializable(class_weight_config), indent=2)}\n\n")

# Train and evaluate each model
for name, model in models.items():
    model_log = [f"\n{'='*50}", f"Model: {name}", f"{'='*50}\n"]
    
    # Log class weight configuration
    if hasattr(model, 'class_weight') and model.class_weight is not None:
        model_log.append(f"Using class weights: {convert_to_serializable(model.class_weight)}\n")
    else:
        model_log.append("No class weights used for this model\n")
    
    # Perform k-fold cross validation with multiple metrics
    cv_results = cross_validate(model, X_tfidf, y, 
                              cv=kf, 
                              scoring=scoring,
                              return_train_score=True)
    
    # Log cross-validation results
    model_log.extend([
        "Cross-validation results:",
        f"Test Accuracy: {cv_results['test_accuracy'].mean():.3f} (+/- {cv_results['test_accuracy'].std() * 2:.3f})",
        f"Test F1 (macro): {cv_results['test_f1_macro'].mean():.3f} (+/- {cv_results['test_f1_macro'].std() * 2:.3f})",
        f"Test F1 (weighted): {cv_results['test_f1_weighted'].mean():.3f} (+/- {cv_results['test_f1_weighted'].std() * 2:.3f})\n"
    ])
    
    # Store predictions and true values for each fold
    predictions = []
    true_values = []
    
    # Train and predict for each fold
    for fold, (train_idx, val_idx) in enumerate(kf.split(X_tfidf), 1):
        X_train_fold = X_tfidf[train_idx]
        X_val_fold = X_tfidf[val_idx]
        y_train_fold = y[train_idx]
        y_val_fold = y[val_idx]
        
        # Train model
        model.fit(X_train_fold, y_train_fold)
        
        # Make predictions
        y_pred_fold = model.predict(X_val_fold)
        
        # Store predictions and true values
        predictions.extend(y_pred_fold)
        true_values.extend(y_val_fold)
        
        # Log fold results
        model_log.extend([
            f"Fold {fold} Results:",
            classification_report(y_val_fold, y_pred_fold, zero_division=0)
        ])
    
    # Create and save confusion matrix
    cm = confusion_matrix(true_values, predictions)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f'Confusion Matrix - {name} (All Folds)')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.savefig(os.path.join(run_dir, 'metrics/confusion_matrices', f'{name.lower()}_cm.png'))
    plt.close()

    # Store model results
    model_results = convert_to_serializable({
        'cross_validation': {
            'accuracy': {'mean': cv_results['test_accuracy'].mean(),
                       'std': cv_results['test_accuracy'].std()},
            'f1_macro': {'mean': cv_results['test_f1_macro'].mean(),
                        'std': cv_results['test_f1_macro'].std()},
            'f1_weighted': {'mean': cv_results['test_f1_weighted'].mean(),
                          'std': cv_results['test_f1_weighted'].std()}
        },
        'confusion_matrix': cm.tolist(),
        'classification_report': classification_report(true_values, predictions, output_dict=True, zero_division=0)
    })

    # Feature importance analysis
    if name in ['Logistic_Regression', 'SVM']:
        model.fit(X_tfidf, y)
        feature_names = vectorizer.get_feature_names_out()
        
        feature_importance = {}
        model_log.append(f"\nTop Features by KC for {name}:")
        
        for i, class_label in enumerate(model.classes_):
            coef = model.coef_[i]
            top_indices = np.argsort(coef)[-10:][::-1]
            
            feature_importance[f'KC_{class_label}'] = {
                feature_names[idx]: float(coef[idx])
                for idx in top_indices
            }
            
            model_log.extend([
                f"\nTop features for KC {class_label}:",
                *[f"{feature_names[idx]}: {coef[idx]:.4f}" for idx in top_indices]
            ])
            
        model_results['feature_importance'] = convert_to_serializable(feature_importance)
        
        # Save feature importance visualization
        plt.figure(figsize=(12, 8))
        plt.bar(range(len(feature_importance[f'KC_{class_label}'])), 
                list(feature_importance[f'KC_{class_label}'].values()))
        plt.xticks(range(len(feature_importance[f'KC_{class_label}'])), 
                   list(feature_importance[f'KC_{class_label}'].keys()), 
                   rotation=45, ha='right')
        plt.title(f'Top Features Importance - {name}')
        plt.tight_layout()
        plt.savefig(os.path.join(run_dir, 'visualizations/feature_importance', f'{name.lower()}_importance.png'))
        plt.close()
    
    all_results[name] = model_results
    
    # Write model logs
    with open(log_file, 'a') as f:
        f.write('\n'.join(model_log))

# Save class distribution analysis
class_distribution = convert_to_serializable({
    'original': dict(zip(*np.unique(y, return_counts=True))),
    'weighted': class_weight_dict
})
with open(os.path.join(run_dir, 'metrics/summary', 'class_distribution.json'), 'w') as f:
    json.dump(class_distribution, f, indent=4)

# Save all results
# 1. Classification reports
for name, results in all_results.items():
    with open(os.path.join(run_dir, 'metrics/classification_reports', f'{name.lower()}_report.json'), 'w') as f:
        json.dump(results['classification_report'], f, indent=4)

# 2. Summary metrics
summary_data = []
for model_name, results in all_results.items():
    summary_data.append({
        'Model': model_name,
        'Class_Weights': 'Yes' if model_name != 'Naive_Bayes' else 'No',
        'Accuracy': f"{results['cross_validation']['accuracy']['mean']:.3f} ± {results['cross_validation']['accuracy']['std']*2:.3f}",
        'F1_Macro': f"{results['cross_validation']['f1_macro']['mean']:.3f} ± {results['cross_validation']['f1_macro']['std']*2:.3f}",
        'F1_Weighted': f"{results['cross_validation']['f1_weighted']['mean']:.3f} ± {results['cross_validation']['f1_weighted']['std']*2:.3f}"
    })

summary_df = pd.DataFrame(summary_data)
summary_df.to_csv(os.path.join(run_dir, 'metrics/summary', 'model_performance.csv'), index=False)

# 3. Full results configuration
with open(os.path.join(run_dir, 'models/configs', 'full_results.json'), 'w') as f:
    json.dump(convert_to_serializable(all_results), f, indent=4)

print(f"\nResults saved in {run_dir}:")
print("├── metrics/")
print("│   ├── confusion_matrices/")
print("│   ├── classification_reports/")
print("│   └── summary/")
print("│       ├── model_performance.csv")
print("│       └── class_distribution.json")
print("├── models/")
print("│   ├── weights/")
print("│   └── configs/")
print("│       ├── class_weights.json")
print("│       └── full_results.json")
print("├── visualizations/")
print("│   └── feature_importance/")
print("└── logs/")
print("    └── training_logs/")


Results saved in ../Results/traditional_ml/weighted/run_20241205_165602:
├── metrics/
│   ├── confusion_matrices/
│   ├── classification_reports/
│   └── summary/
│       ├── model_performance.csv
│       └── class_distribution.json
├── models/
│   ├── weights/
│   └── configs/
│       ├── class_weights.json
│       └── full_results.json
├── visualizations/
│   └── feature_importance/
└── logs/
    └── training_logs/


In [1]:
from ml_experiments import run_ml_experiment
import pandas as pd

data_path = '../Data/Cleaned_Mel_CPS_19_Dataset.csv'
data = pd.read_csv(data_path)

# Run experiments with different configurations
run_dir_1 = run_ml_experiment(data, merge_codes=[[4,5]], use_weights=True)
print(f"Experiment 1 results saved in: {run_dir_1}")

run_dir_2 = run_ml_experiment(data, use_weights=False)
print(f"Experiment 2 results saved in: {run_dir_2}")

# run_dir_3 = run_ml_experiment(data, merge_codes=[[4,5], [1,2]], use_weights=True)
# print(f"Experiment 3 results saved in: {run_dir_3}") 

/Users/amin/Library/CloudStorage/GoogleDrive-masamadi@uci.edu/Shared drives/La(LA)LAB/Research Projects/kc-coding/Code/ml_experiments.py:94: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'KC_4_5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df['KC'].isin(codes_to_merge), 'KC'] = new_kc
/Users/amin/Library/CloudStorage/GoogleDrive-masamadi@uci.edu/Shared drives/La(LA)LAB/Research Projects/kc-coding/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/amin/Library/CloudStorage/GoogleDrive-masamadi@uci.edu/Shared drives/La(LA)LAB/Research Projects/kc-coding/.venv/lib/python3.12/site-packages/sklea

Experiment 1 results saved in: ../Results/traditional_ml/weighted_merged/run_20241209_160425


/Users/amin/Library/CloudStorage/GoogleDrive-masamadi@uci.edu/Shared drives/La(LA)LAB/Research Projects/kc-coding/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/amin/Library/CloudStorage/GoogleDrive-masamadi@uci.edu/Shared drives/La(LA)LAB/Research Projects/kc-coding/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/amin/Library/CloudStorage/GoogleDrive-masamadi@uci.edu/Shared drives/La(LA)LAB/Research Projects/kc-coding/.venv/lib/python3.12/site-packages

Experiment 2 results saved in: ../Results/traditional_ml/unweighted/run_20241209_160508
